In [30]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [33]:
MODEL_PATH = '/models/nlp_models'

In [34]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Text preprocessing functions**

In [35]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [105]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 25000)

In [112]:
MAX_LEN = 170
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:20000]  # make dataset size 20000 for ease of computation

In [113]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [116]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 39


### **building the dataset**

In [117]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [118]:
shuffle_pairs = create_shuffles(sents_data, 8)

In [120]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' اوهم اکنون درحالی که جراحات سختی براثر سوختگی برداشته است دریکی ازبیمارستان\u200cهای اسکاتلند تحت حفاظت قراردارد',
 {'اسکاتلند است دریکی جراحات قراردارد درحالی حفاظت براثر سختی سوختگی ازبیمارستان\u200cهای برداشته که اوهم اکنون تحت',
  'اسکاتلند دریکی حفاظت برداشته تحت قراردارد است که درحالی براثر ازبیمارستان\u200cهای سختی سوختگی اکنون اوهم جراحات',
  'اسکاتلند دریکی درحالی براثر برداشته قراردارد اکنون حفاظت جراحات که سختی تحت سوختگی است ازبیمارستان\u200cهای اوهم',
  'اوهم جراحات حفاظت درحالی دریکی ازبیمارستان\u200cهای سوختگی که اکنون است تحت برداشته براثر سختی قراردارد اسکاتلند',
  'برداشته دریکی براثر اسکاتلند اکنون تحت جراحات است قراردارد که سوختگی اوهم درحالی سختی حفاظت ازبیمارستان\u200cهای',
  'برداشته که سوختگی براثر حفاظت جراحات ازبیمارستان\u200cهای دریکی اکنون قراردارد درحالی اسکاتلند است اوهم تحت سختی',
  'تحت درحالی دریکی برداشته قراردارد اسکاتلند ازبیمارستان\u200cهای سوختگی اوهم است سختی براثر جراحات که حفاظت اکنون',
  'قراردارد است تحت جراحات سختی که سوختگی اسکاتلند اکنون

In [121]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [122]:
df = create_dataframe(shuffle_pairs)
df

,Shuffled,Original
0,کنند از شو این کار تا طرف‌ها بی بیش همه وزارت ...,لئو جیان شو سخنگوی وزارت امور خارجه چین امروز...
1,تا بی طرف‌ها از سه نشود خارجه چین جیان سخنگوی ...,لئو جیان شو سخنگوی وزارت امور خارجه چین امروز...
2,کار سه از با خواست منطقه سخنگوی چین حل بیش خار...,لئو جیان شو سخنگوی وزارت امور خارجه چین امروز...
3,امور طرف‌ها یکدیگر از منطقه خواست چین خارجه جی...,لئو جیان شو سخنگوی وزارت امور خارجه چین امروز...
4,کار این لئو تا بی سه سخنگوی ثبات خواست امور حل...,لئو جیان شو سخنگوی وزارت امور خارجه چین امروز...
...,...,...
177612,ذره حقوق دایان Dianne گروه مشترک جدید که مامور...,سناتور دایان فاینشتاین Dianne Feinstein رئیس ...
177613,ماموریت این گروه وخیم‌ترین ذره جدید فاینشتاین ...,سناتور دایان فاینشتاین Dianne Feinstein رئیس ...
177614,را گروه بین که Feinstein رئیس قراردهد Dianne ذ...,سناتور دایان فاینشتاین Dianne Feinstein رئیس ...
177615,Feinstein گروه سناتور ماموریت نقض این که مشترک...,سناتور دایان فاینشتاین Dianne Feinstein رئیس ...


In [123]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [124]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [125]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [126]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [127]:
MAX_SEQ_LEN = 45  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN)
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN)

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN)
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN)

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN)
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN)

In [128]:
X_train_decoder[:2], X_test_decoder[:2]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 24233, 11550,  7018, 11551,
           122,   109,   110,   128,   112,    44,    46,    59,     5,
           363,  4469,   332,    12,   374,   481,    10,   569,   167,
            94,    54,   117,   119,     5,     9,   179,  1317,  1540],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 24233, 11550,  7018, 11551,
           122,   109,   110,   128,   112,    44,    46,    59,     5,
           363,  4469,   332,    12,   374,   481,    10,   569,   167,
            94,    54,   117,   119,     5,     9,   179,  1317,  1540]],
       dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 24233,    20,  1882,
            27,   164,   259,   123,     5,  1119,    19,     6,   258,
            28,     8,     4,    85,    

In [129]:
X_train[0], Y_train[0]

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    94,     5, 11551,
            9,   167,    54,  4469,   179,   119,   363,   109,    12,
         1317,   332,  7018,    59,     5, 11550,    10,   569,   128,
         1540,   122,    46,   112,   374,   481,   117,    44,   110],
       dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 11550,  7018, 11551,
          122,   109,   110,   128,   112,    44,    46,    59,     5,
          363,  4469,   332,    12,   374,   481,    10,   569,   167,
           94,    54,   117,   119,     5,     9,   179,  1317,  1540],
       dtype=int32))

In [130]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

24233

###**Encoder-Decoder network**

In [132]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1,output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, dropout=0.2, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, dropout=0.2, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_14 (Embedding)       (None, None, 32)     775488      ['input_15[0][0]']               
                                                                                                  
 embedding_15 (Embedding)       (None, None, 32)     775520      ['input_16[0][0]']               
                                                                                            

In [133]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=30, validation_split=0.1, batch_size=512)

Epoch 1/30
250/250 [==============================] - 193s 741ms/step - loss: 3.3845 - accuracy: 0.0804 - val_loss: 3.0433 - val_accuracy: 0.0845
Epoch 2/30
250/250 [==============================] - 181s 721ms/step - loss: 2.8460 - accuracy: 0.0972 - val_loss: 2.6607 - val_accuracy: 0.1264
Epoch 3/30
250/250 [==============================] - 181s 725ms/step - loss: 2.3881 - accuracy: 0.1844 - val_loss: 2.1692 - val_accuracy: 0.2953
Epoch 4/30
250/250 [==============================] - 180s 720ms/step - loss: 1.8690 - accuracy: 0.3899 - val_loss: 1.6787 - val_accuracy: 0.5359
Epoch 5/30
250/250 [==============================] - 180s 721ms/step - loss: 1.4353 - accuracy: 0.5807 - val_loss: 1.3427 - val_accuracy: 0.6669
Epoch 6/30
250/250 [==============================] - 181s 724ms/step - loss: 1.1390 - accuracy: 0.6783 - val_loss: 1.1246 - val_accuracy: 0.7286
Epoch 7/30
250/250 [==============================] - 181s 725ms/step - loss: 0.9333 - accuracy: 0.7336 - val_loss: 0.9692 -

In [140]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_split=0.1, batch_size=512)

Epoch 1/10
250/250 [==============================] - 198s 755ms/step - loss: 0.0119 - accuracy: 0.9979 - val_loss: 0.2859 - val_accuracy: 0.9453
Epoch 2/10
250/250 [==============================] - 184s 737ms/step - loss: 0.0101 - accuracy: 0.9981 - val_loss: 0.2829 - val_accuracy: 0.9457
Epoch 3/10
250/250 [==============================] - 182s 729ms/step - loss: 0.0087 - accuracy: 0.9983 - val_loss: 0.2789 - val_accuracy: 0.9461
Epoch 4/10
250/250 [==============================] - 183s 734ms/step - loss: 0.0079 - accuracy: 0.9983 - val_loss: 0.2774 - val_accuracy: 0.9465
Epoch 5/10
250/250 [==============================] - 183s 733ms/step - loss: 0.0071 - accuracy: 0.9985 - val_loss: 0.2757 - val_accuracy: 0.9468
Epoch 6/10
250/250 [==============================] - 183s 734ms/step - loss: 0.0064 - accuracy: 0.9986 - val_loss: 0.2740 - val_accuracy: 0.9471
Epoch 7/10
250/250 [==============================] - 183s 733ms/step - loss: 0.0058 - accuracy: 0.9987 - val_loss: 0.2725 -

**almost 100% accuracy on training data after 30 epochs**

In [141]:
model.save(MODEL_PATH)

INFO:tensorflow:Assets written to: /models/nlp_models/assets


INFO:tensorflow:Assets written to: /models/nlp_models/assets


###**Simple seq2seq network**

**model evaluation on test data**

In [142]:
model = keras.models.load_model(MODEL_PATH)

In [143]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[ 1258,  1258,  1258,  1258,  1258,  1258,  1258,  1258,  1258,
          1258,  1258,  1258,  1258,  1258,  1258,     0,    20,  1882,
            27,   164,   259,   123,     5,  1119,    19,     6,   258,
            28,     8,     4,    85,    88,   419,    13,     6,     5,
           457,     7,    20, 20473,    19,   406,   205,   894,  5517]]),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,    20,  1882,
            27,   164,   259,   123,     5,  1119,    19,     6,   258,
            28,     8,     4,    85,    88,   419,    13,     6,     5,
           457,     7,    20, 21638,    19,   406,   205,   894,  5517]],
       dtype=int32))

In [144]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=512)
print("test loss, test acc:", results)

Evaluate on test data
70/70 [==============================] - 24s 308ms/step - loss: 0.2791 - accuracy: 0.9476
test loss, test acc: [0.2790781855583191, 0.9475856423377991]


###**Rebuild sentences**

In [200]:
def delete_paddings(array):
  for i, arr in array:
    arr = np.trim_zeros(arr)
  return array


In [201]:
arr = np.array([[0, 0, 5, 5, 7], [0, 0, 1, 2, 5]])

delete_paddings(arr)

array([[0, 0, 5, 5, 7],
       [0, 0, 1, 2, 5]])

In [167]:
def rebuild_test_sentences():
  Y_preds = model.predict([X_test[:10], X_test_decoder[:10]])
  encoded_argmax  = np.argmax(Y_preds, axis=-1)
  print(encoded_argmax.shape)
  
  original = tokenizer.sequences_to_texts(Y_test[:10])
  preds = tokenizer.sequences_to_texts(encoded_argmax[:10])
  return preds, original
 

rebuild_test_sentences()

(10, 45)


(['پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند <UNK> ایران ارزیابی دو مقام ارشد آمریکائی از اوضاع عراق را رد کرده است و میگوید گزارش آنان آمریکا را از آنچه که ایران ازتعلیق عراق توصیف میکند بیرون نمی\u200cآورد',
  'پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند <UNK> ایران ارزیابی دو مقام ارشد آمریکائی از اوضاع عراق را رد کرده است و میگوید گزارش آنان آمریکا را از آنچه که ایران ازتعلیق عراق توصیف میکند بیرون نمی\u200cآورد',
  'پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند <UNK> ایران ارزیابی دو مقام ارشد آمریکائی از اوضاع عراق را رد کرده است و میگوید گزارش آنان آمریکا را از آنچه که ایران ازتعلیق عراق توصیف میکند بیرون نمی\u200cآورد',
  'پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختند پرداختن